# AutoML

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

## Table of Contents

* [Setup and Preprocessing](#setup)  
* [Compare Models](#compare)  
* [Create Model](#create)  
* [Tune Model](#tune)  
* [Evaluate Model](#evaluate)  
* [Finalize and Store Model](#finalize_and_store)  

## Imports and Global Settings

In [1]:
import sys
import mlflow
import pandas as pd
from sqlalchemy import create_engine
from pycaret import regression as py_reg
from pycaret import classification as py_cls

sys.path.append('../')
from passkeys import RDS_ENDPOINT, RDS_PASSWORD

# Pandas Settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5


mlflow.set_tracking_uri('file:/home/jeff/Documents/Data_Science_Projects/NBA_Betting/models/AutoML')

## Database Connection

In [2]:
username = 'postgres'
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = 'nba_betting'
port = '5432'

connection = create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

### Datasets

In [3]:
df = pd.read_sql_table('model_training_data', connection)

### Restrict to previous years games only

In [4]:
df = df[df['league_year_end'] != 23]

In [5]:
df.sort_values('game_id', ascending=False).head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consensus_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_vla,offrtg_vla_std,defrtg_rank,defrtg_vla,defrtg_vla_std,netrtg_rank,netrtg_vla,netrtg_vla_std,ast_pct_rank,ast_pct_vla,ast_pct_vla_std,ast_v_tov_rank,ast_v_tov_vla,ast_v_tov_vla_std,ast_ratio_rank,ast_ratio_vla,...,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,opp_fg_pct_atb3_rank,opp_fg_pct_atb3_vla,opp_fg_pct_atb3_vla_std

<a id='basic_data_overview'></a>

## Basic Data Overview

In [6]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 0 to 9353
Data columns (total 1223 columns):
 #     Column                                 Non-Null Count  Dtype  
---    ------                                 --------------  -----  
 0     game_id                                9200 non-null   object 
 1     CLS_TARGET_home_margin_GT_home_spread  9200 non-null   bool   
 2     REG_TARGET_actual_home_margin          9200 non-null   float64
 3     home_team_num                          9200 non-null   int64  
 4     away_team_num                          9200 non-null   int64  
 5     league_year_end                        9200 non-null   int64  
 6     fd_line_home                           15 non-null     float64
 7     dk_line_home                           14 non-null     float64
 8     covers_consensus_home                  15 non-null     float64
 9     home_spread                            9200 non-null   float64
 10    gp                                     9171 

In [7]:
df.head()

,game_id,CLS_TARGET_home_margin_GT_home_spread,REG_TARGET_actual_home_margin,home_team_num,away_team_num,league_year_end,fd_line_home,dk_line_home,covers_consensus_home,home_spread,gp,win,loss,w_pct,mins,pts,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,tov,stl,blk,blka,pf,pfd,p_m,gp_rank,gp_vla,gp_vla_std,win_rank,win_vla,win_vla_std,loss_rank,loss_vla,loss_vla_std,w_pct_rank,w_pct_vla,w_pct_vla_std,mins_rank,mins_vla,mins_vla_std,pts_rank,pts_vla,pts_vla_std,fgm_rank,fgm_vla,fgm_vla_std,fga_rank,fga_vla,fga_vla_std,fg_pct_rank,fg_pct_vla,fg_pct_vla_std,fg3m_rank,fg3m_vla,fg3m_vla_std,fg3a_rank,fg3a_vla,fg3a_vla_std,fg3_pct_rank,fg3_pct_vla,fg3_pct_vla_std,ftm_rank,ftm_vla,ftm_vla_std,fta_rank,fta_vla,fta_vla_std,ft_pct_rank,ft_pct_vla,ft_pct_vla_std,oreb_rank,oreb_vla,oreb_vla_std,dreb_rank,dreb_vla,dreb_vla_std,reb_rank,reb_vla,reb_vla_std,ast_rank,ast_vla,ast_vla_std,tov_rank,tov_vla,tov_vla_std,stl_rank,stl_vla,stl_vla_std,blk_rank,blk_vla,blk_vla_std,blka_rank,blka_vla,blka_vla_std,pf_rank,pf_vla,pf_vla_std,pfd_rank,pfd_vla,pfd_vla_std,p_m_rank,p_m_vla,p_m_vla_std,gp_opp,win_opp,loss_opp,w_pct_opp,mins_opp,pts_opp,fgm_opp,fga_opp,fg_pct_opp,fg3m_opp,fg3a_opp,fg3_pct_opp,ftm_opp,fta_opp,ft_pct_opp,oreb_opp,dreb_opp,reb_opp,ast_opp,tov_opp,stl_opp,blk_opp,blka_opp,pf_opp,pfd_opp,p_m_opp,gp_rank_opp,gp_vla_opp,gp_vla_std_opp,win_rank_opp,win_vla_opp,win_vla_std_opp,loss_rank_opp,loss_vla_opp,loss_vla_std_opp,w_pct_rank_opp,w_pct_vla_opp,w_pct_vla_std_opp,mins_rank_opp,mins_vla_opp,mins_vla_std_opp,pts_rank_opp,pts_vla_opp,pts_vla_std_opp,fgm_rank_opp,fgm_vla_opp,fgm_vla_std_opp,fga_rank_opp,fga_vla_opp,fga_vla_std_opp,fg_pct_rank_opp,fg_pct_vla_opp,fg_pct_vla_std_opp,fg3m_rank_opp,fg3m_vla_opp,fg3m_vla_std_opp,fg3a_rank_opp,fg3a_vla_opp,fg3a_vla_std_opp,fg3_pct_rank_opp,fg3_pct_vla_opp,fg3_pct_vla_std_opp,ftm_rank_opp,ftm_vla_opp,ftm_vla_std_opp,fta_rank_opp,fta_vla_opp,fta_vla_std_opp,ft_pct_rank_opp,ft_pct_vla_opp,ft_pct_vla_std_opp,oreb_rank_opp,oreb_vla_opp,oreb_vla_std_opp,dreb_rank_opp,dreb_vla_opp,dreb_vla_std_opp,reb_rank_opp,reb_vla_opp,reb_vla_std_opp,ast_rank_opp,ast_vla_opp,ast_vla_std_opp,tov_rank_opp,tov_vla_opp,tov_vla_std_opp,stl_rank_opp,stl_vla_opp,stl_vla_std_opp,blk_rank_opp,blk_vla_opp,blk_vla_std_opp,blka_rank_opp,blka_vla_opp,blka_vla_std_opp,pf_rank_opp,pf_vla_opp,pf_vla_std_opp,pfd_rank_opp,pfd_vla_opp,pfd_vla_std_opp,p_m_rank_opp,p_m_vla_opp,p_m_vla_std_opp,offrtg,defrtg,netrtg,ast_pct,ast_v_tov,ast_ratio,oreb_pct,dreb_pct,reb_pct,tov_pct,efg_pct,ts_pct,pace,pie,poss,offrtg_rank,offrtg_vla,offrtg_vla_std,defrtg_rank,defrtg_vla,defrtg_vla_std,netrtg_rank,netrtg_vla,netrtg_vla_std,ast_pct_rank,ast_pct_vla,ast_pct_vla_std,ast_v_tov_rank,ast_v_tov_vla,ast_v_tov_vla_std,ast_ratio_rank,ast_ratio_vla,...,opp_fga_atb3,opp_fg_pct_atb3,opp_fgm_ra_rank,opp_fgm_ra_vla,opp_fgm_ra_vla_std,opp_fga_ra_rank,opp_fga_ra_vla,opp_fga_ra_vla_std,opp_fg_pct_ra_rank,opp_fg_pct_ra_vla,opp_fg_pct_ra_vla_std,opp_fgm_paint_rank,opp_fgm_paint_vla,opp_fgm_paint_vla_std,opp_fga_paint_rank,opp_fga_paint_vla,opp_fga_paint_vla_std,opp_fg_pct_paint_rank,opp_fg_pct_paint_vla,opp_fg_pct_paint_vla_std,opp_fgm_mr_rank,opp_fgm_mr_vla,opp_fgm_mr_vla_std,opp_fga_mr_rank,opp_fga_mr_vla,opp_fga_mr_vla_std,opp_fg_pct_mr_rank,opp_fg_pct_mr_vla,opp_fg_pct_mr_vla_std,opp_fgm_lc3_rank,opp_fgm_lc3_vla,opp_fgm_lc3_vla_std,opp_fga_lc3_rank,opp_fga_lc3_vla,opp_fga_lc3_vla_std,opp_fg_pct_lc3_rank,opp_fg_pct_lc3_vla,opp_fg_pct_lc3_vla_std,opp_fgm_rc3_rank,opp_fgm_rc3_vla,opp_fgm_rc3_vla_std,opp_fga_rc3_rank,opp_fga_rc3_vla,opp_fga_rc3_vla_std,opp_fg_pct_rc3_rank,opp_fg_pct_rc3_vla,opp_fg_pct_rc3_vla_std,opp_fgm_c3_rank,opp_fgm_c3_vla,opp_fgm_c3_vla_std,opp_fga_c3_rank,opp_fga_c3_vla,opp_fga_c3_vla_std,opp_fg_pct_c3_rank,opp_fg_pct_c3_vla,opp_fg_pct_c3_vla_std,opp_fgm_atb3_rank,opp_fgm_atb3_vla,opp_fgm_atb3_vla_std,opp_fga_atb3_rank,opp_fga_atb3_vla,opp_fga_atb3_vla_std,opp_fg_pct_atb3_rank,opp_fg_pct_atb3_vla,opp_fg_pct_atb3_vla_std

## PyCaret - Regression

<a id=setup></a>

### Setup and Preprocessing

In [8]:
target = ['REG_TARGET_actual_home_margin']
drop_features = ['fd_line_home', 'dk_line_home', 'covers_consensus_home', 'game_id',
                 'REG_TARGET_actual_home_margin', 'CLS_TARGET_home_margin_GT_home_spread']
main_features = ['home_team_num', 'away_team_num', 'home_spread',
                 'league_year_end', 'day_of_season', 'elo1_pre',
                 'elo2_pre', 'elo_prob1', 'elo_prob2']
rank_features = [feature for feature in list(df) if 'rank' in feature]
vla_std_features = [feature for feature in list(df) if 'vla_std' in feature]
                 
other_features = [feature for feature in list(df) if feature not in target + main_features + drop_features]

features_to_use = target + main_features + rank_features + vla_std_features

In [9]:
model_ready_df = df[features_to_use]

In [10]:
model_ready_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 0 to 9353
Data columns (total 614 columns):
 #    Column                              Dtype  
---   ------                              -----  
 0    REG_TARGET_actual_home_margin       float64
 1    home_team_num                       int64  
 2    away_team_num                       int64  
 3    home_spread                         float64
 4    league_year_end                     int64  
 5    day_of_season                       int64  
 6    elo1_pre                            float64
 7    elo2_pre                            float64
 8    elo_prob1                           float64
 9    elo_prob2                           float64
 10   gp_rank                             float64
 11   win_rank                            float64
 12   loss_rank                           float64
 13   w_pct_rank                          float64
 14   mins_rank                           float64
 15   pts_rank                            

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [11]:
setup_params = {'log_experiment': True,
                'log_profile': False,
                'log_plots': False,
                'experiment_name': 'NBA_Betting_REG_vlastd_rank',
                'data': model_ready_df,
                'target': 'REG_TARGET_actual_home_margin',
                'train_size': 0.7,
                'preprocess': True,
                'normalize': False,        # zscore
                'transformation': False,   # yeo-johnson power transform to make data more Gaussian
                'remove_outliers': False,  # using SVD
                'remove_multicollinearity': False,
                'polynomial_features': False,
                'trigonometry_features': False,
                'feature_interaction': False,
                'feature_ratio': False,
                'feature_selection': False,
                'feature_selection_threshold': 0.8,
                'pca': False,
                'pca_components': 10,
                'numeric_features': [],
                'ignore_features': []
               }

In [12]:
nba_betting_regression = py_reg.setup(**setup_params)

,Description,Value
0,session_id,6710
1,Target,REG_TARGET_actual_home_margin
2,Original Data,"(9200, 614)"
3,Missing Values,True
4,Numeric Features,612
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(6439, 600)"


<a id=compare></a>

### Compare Models

In [13]:
best_3_models = py_reg.compare_models(n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,9.8116,158.6576,12.5920,0.2076,1.0271,1.1514,0.0670
en,Elastic Net,9.8488,159.6328,12.6304,0.2028,1.0411,1.1620,0.0820
omp,Orthogonal Matching Pursuit,9.9479,162.5818,12.7460,0.1882,1.0424,1.1970,0.0490
br,Bayesian Ridge,10.0285,164.4606,12.8188,0.1790,1.1361,1.1354,0.8240
ridge,Ridge Regression,10.3552,174.9087,13.2200,0.1264,1.0743,1.2987,0.0530
lr,Linear Regression,10.3770,175.5727,13.2453,0.1231,1.0735,1.3022,0.4770
llar,Lasso Least Angle Regression,11.3173,200.9297,14.1677,-0.0023,1.3215,1.0328,0.0480
dummy,Dummy Regressor,11.3173,200.9297,14.1677,-0.0023,1.3215,1.0328,0.3810
par,Passive Aggressive Regressor,16.3969,394.5884,19.4516,-0.9751,1.0746,2.6177,0.1040
lar,Least Angle Regression,1424.3192,34585515.1843,3192.3743,-181193.1609,3.0140,260.9375,0.1420


<a id=create></a>

### Create Selected Model

In [14]:
model = py_reg.create_model('lasso')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,9.4836,151.8080,12.3210,0.2333,1.0213,1.1120
1,9.9547,165.5380,12.8662,0.2199,1.0759,1.1954
2,9.3507,142.8759,11.9531,0.1962,1.0099,1.1179
3,9.6145,151.0143,12.2888,0.1902,1.0331,1.1709
4,10.2238,166.9926,12.9226,0.1992,1.0662,1.2730
5,9.8560,162.9924,12.7668,0.2501,0.9841,1.0944
6,10.1081,168.8246,12.9933,0.2181,1.0438,1.1162
7,9.8728,161.4861,12.7077,0.2071,1.0440,1.1532
8,9.9497,160.1189,12.6538,0.1920,0.9752,1.1353
9,9.7018,154.9247,12.4469,0.1700,1.0172,1.1457


<a id=tune></a>

### Tune Selected Model

In [15]:
tuned_model = py_reg.tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,9.5372,153.2732,12.3804,0.2259,1.0237,1.1016
1,9.9849,166.5849,12.9068,0.2149,1.0662,1.1816
2,9.3243,141.9395,11.9138,0.2014,1.0060,1.0980
3,9.6305,151.8405,12.3224,0.1858,1.0399,1.1533
4,10.1831,166.8054,12.9153,0.2001,1.0566,1.2392
5,9.8231,162.0642,12.7304,0.2544,0.9863,1.0812
6,10.2047,170.9272,13.0739,0.2083,1.0552,1.1143
7,9.8519,160.8083,12.6810,0.2104,1.0282,1.1315
8,9.8945,158.6319,12.5949,0.1995,0.9858,1.1060
9,9.7397,155.6034,12.4741,0.1664,1.0316,1.1326


<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [16]:
py_reg.evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [17]:
# py_reg.interpret_model(tuned_model)

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [18]:
final_model = py_reg.finalize_model(tuned_model)

In [19]:
# py_reg.save_model(final_model, '../models/AutoML/vlastd_Rank_Lasso_Reg_PyCaret')

In [20]:
# !mlflow ui

## Classification

<a id=setup></a>

### Setup and Preprocessing

In [21]:
target = ['CLS_TARGET_home_margin_GT_home_spread']
drop_features = ['fd_line_home', 'dk_line_home', 'covers_consensus_home', 'game_id',
                 'REG_TARGET_actual_home_margin', 'CLS_TARGET_home_margin_GT_home_spread']
main_features = ['home_team_num', 'away_team_num', 'home_spread',
                 'league_year_end', 'day_of_season', 'elo1_pre',
                 'elo2_pre', 'elo_prob1', 'elo_prob2']
rank_features = [feature for feature in list(df) if 'rank' in feature]
vla_std_features = [feature for feature in list(df) if 'vla_std' in feature]
                 
other_features = [feature for feature in list(df) if feature not in target + main_features + drop_features]

features_to_use = target + main_features + rank_features + vla_std_features

In [22]:
model_ready_df = df[features_to_use]

In [23]:
model_ready_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 0 to 9353
Columns: 614 entries, CLS_TARGET_home_margin_GT_home_spread to contested_shots_vla_std_opp
dtypes: bool(1), float64(609), int64(4)
memory usage: 43.1 MB


The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [24]:
setup_params = {'log_experiment': True,
                'log_profile': False,
                'log_plots': False,
                'experiment_name': 'NBA_Betting_CLS_vlastd_rank',
                'data': model_ready_df,
                'target': 'CLS_TARGET_home_margin_GT_home_spread',
                'train_size': 0.7,
                'preprocess': True,
                'normalize': False,        # zscore
                'transformation': False,   # yeo-johnson power transform to make data more Gaussian
                'remove_outliers': False,  # using SVD
                'remove_multicollinearity': False,
                'polynomial_features': False,
                'trigonometry_features': False,
                'feature_interaction': False,
                'feature_ratio': False,
                'feature_selection': False,
                'feature_selection_threshold': 0.8,
                'pca': False,
                'pca_components': 10,
                'numeric_features': ['league_year_end'],
                'ignore_features': []
               }

In [25]:
nba_betting_classification = py_cls.setup(**setup_params)

,Description,Value
0,session_id,8732
1,Target,CLS_TARGET_home_margin_GT_home_spread
2,Target Type,Binary
3,Label Encoded,"False: 0, True: 1"
4,Original Data,"(9200, 614)"
5,Missing Values,True
6,Numeric Features,613
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


<a id=compare></a>

### Compare Models

In [26]:
best_3_models = py_cls.compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.5159,0.0000,0.4895,0.5045,0.4967,0.0307,0.0307,0.0620
lr,Logistic Regression,0.5150,0.5111,0.4829,0.5041,0.4929,0.0286,0.0287,3.0780
nb,Naive Bayes,0.5016,0.5007,0.5083,0.4901,0.4988,0.0036,0.0036,0.0450
knn,K Neighbors Classifier,0.4988,0.4965,0.4841,0.4871,0.4855,-0.0030,-0.0030,1.9650
svm,SVM - Linear Kernel,0.4967,0.0000,0.5505,0.4120,0.3961,-0.0043,-0.0055,0.7080
dt,Decision Tree Classifier,0.4886,0.4883,0.4752,0.4769,0.4760,-0.0234,-0.0234,1.7750


<a id=create></a>

### Create Selected Model

In [27]:
model = py_cls.create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5217,0.5046,0.4540,0.5125,0.4815,0.0407,0.0410
1,0.5280,0.5222,0.4984,0.5182,0.5081,0.0547,0.0547
2,0.5543,0.5445,0.5048,0.5483,0.5256,0.1068,0.1071
3,0.5155,0.5104,0.4667,0.5052,0.4851,0.0290,0.0291
4,0.5140,0.4993,0.4667,0.5034,0.4843,0.0260,0.0260
5,0.4783,0.4761,0.4317,0.4642,0.4474,-0.0455,-0.0456
6,0.5062,0.5019,0.4809,0.4935,0.4871,0.0112,0.0112
7,0.4845,0.4928,0.4809,0.4719,0.4763,-0.0312,-0.0312
8,0.5155,0.5226,0.5350,0.5030,0.5185,0.0320,0.0320
9,0.5319,0.5362,0.5096,0.5212,0.5153,0.0628,0.0628


<a id=tune></a>

### Tune Selected Model

In [28]:
tuned_model = py_cls.tune_model(model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5202,0.5061,0.4889,0.5099,0.4992,0.0391,0.0391
1,0.5326,0.5222,0.5429,0.5213,0.5319,0.0656,0.0656
2,0.5342,0.5435,0.5238,0.5238,0.5238,0.0679,0.0679
3,0.5264,0.5107,0.5048,0.5162,0.5104,0.0519,0.0519
4,0.5124,0.4998,0.4889,0.5016,0.4952,0.0239,0.0239
5,0.4783,0.4744,0.4730,0.4671,0.4700,-0.0437,-0.0437
6,0.5016,0.4922,0.5064,0.4892,0.4977,0.0033,0.0033
7,0.4720,0.4890,0.5032,0.4620,0.4817,-0.0543,-0.0545
8,0.5171,0.5156,0.5573,0.5043,0.5295,0.0360,0.0362
9,0.5272,0.5352,0.5478,0.5150,0.5309,0.0553,0.0554


<a id=evaluate></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [29]:
py_cls.evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [30]:
# py_cls.interpret_model(tuned_model)

<a id=finalize_and_store></a>

### Model Finalization and Storage

In [31]:
final_model = py_cls.finalize_model(tuned_model)

In [33]:
# py_cls.save_model(final_model, '../models/AutoML/vlastd_rank_LR_CLS_PyCaret')

In [ ]:
#!mlflow ui